In [110]:
import sklearn
import re
import numpy as np
import scipy as sp
import pandas as pd
import glove_solution
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [2]:
# Import all vector words
all_words = np.load('embeddings.npy')

In [141]:
# Import vocab and transform it into an array
vocab_cut = pd.read_fwf('vocab_cut.txt', sep = "\t", header = None, error_bad_lines=False)
vocab_cut.columns = ['word']
voc = vocab_cut.values
voc

array([['<user>'],
       ['!'],
       ['i'],
       ...,
       ['#1dson'],
       ['#18'],
       ['#10fac']], dtype=object)

In [4]:
# Load train set positiv and train set negativ
tweet_pos = pd.read_csv('data/train_pos.txt', sep = '\t', header = None)
tweet_neg = pd.read_csv('data/train_neg.txt', sep = '\t', header = None)
tweet_test = pd.read_csv('data/test_data.txt', sep = '\t', header = None)

In [5]:
tweet_pos.head()

,0
0,<user> i dunno justin read my mention or not ....
1,"because your logic is so dumb , i won't even c..."
2,<user> just put casper in a box ! looved the...
3,<user> <user> thanks sir > > don't trip lil ma...
4,visiting my brother tmr is the bestest birthda...


In [6]:
tweet_neg.head()

,0
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,glad i dot have taks tomorrow ! ! #thankful #s...
2,1-3 vs celtics in the regular season = were fu...
3,<user> i could actually kill that girl i'm so ...
4,<user> <user> <user> i find that very hard to ...


In [7]:
tweet_test.head()

,0
0,"1,sea doo pro sea scooter ( sports with the po..."
1,"2,<user> shucks well i work all week so now i ..."
2,"3,i cant stay away from bug thats my baby"
3,"4,<user> no ma'am ! ! ! lol im perfectly fine ..."
4,"5,whenever i fall asleep watching the tv , i a..."


In [8]:
def tweet_to_vect(tweet):
    """ Map a tweet to a vector using the
        vector of each word that is present
        in the tweet"""
    word_array = re.findall(r'\w+', tweet)
    vect = np.zeros(20)
    for word in word_array:
        i = np.argwhere(voc == word)
        if(i.shape[0] != 0): 
            vect += all_words[i[0][0]]
    return vect/len(word_array)

In [9]:
tweet_to_vect('i love machine learning')

array([ 0.08483506,  0.18905336,  0.13301682,  0.28761705,  0.15026169,
        0.16017279, -0.12708856, -0.16646328, -0.28536286,  0.23276146,
       -0.03472253, -0.06886545, -0.33791882, -0.17811641, -0.39209849,
       -0.05214223,  0.27099058, -0.06285391,  0.33469796,  0.25988553])

In [10]:
tweet_pos['vect'] = ""
tweet_pos['pred'] = 1
tweet_neg['vect'] = ""
tweet_neg['pred'] = -1

tweet_pos.columns = ['tweet', 'vect', 'pred']
tweet_neg.columns = ['tweet', 'vect', 'pred']

In [11]:
tweet_pos.vect = tweet_pos.tweet.apply(lambda x: tweet_to_vect(x))
tweet_neg.vect = tweet_neg.tweet.apply(lambda x: tweet_to_vect(x))

In [12]:
tweet_pos.head()

,tweet,vect,pred
0,<user> i dunno justin read my mention or not ....,"[0.12187350126213467, 0.2392204874081336, 0.17...",1
1,"because your logic is so dumb , i won't even c...","[0.029361259698287817, 0.2845059508864009, 0.1...",1
2,<user> just put casper in a box ! looved the...,"[0.03723710238158048, 0.010057370590752113, 0....",1
3,<user> <user> thanks sir > > don't trip lil ma...,"[0.01164429224360294, -0.1313781541898918, 0.1...",1
4,visiting my brother tmr is the bestest birthda...,"[0.12157701280012964, 0.07784914951046543, 0.0...",1


In [13]:
tweet_neg.head()

,tweet,vect,pred
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,"[0.01879816550451843, 0.003126974568930025, 0....",-1
1,glad i dot have taks tomorrow ! ! #thankful #s...,"[0.2500810171748917, 0.05640994032588838, 0.18...",-1
2,1-3 vs celtics in the regular season = were fu...,"[0.08948027579672979, 0.1839799238063547, 0.22...",-1
3,<user> i could actually kill that girl i'm so ...,"[0.11785501831281628, 0.19467555676440992, 0.3...",-1
4,<user> <user> <user> i find that very hard to ...,"[0.08321729153265517, -0.12035350613718467, 0....",-1


In [120]:
# Map all -1 to 0
def neg_to_zero(array):
    ret = np.zeros(len(array))
    for i in range(len(array)):
        if(array[i] == 1):
            ret[i] = 1
    return ret

In [214]:
# Map all 0 to -1
def zero_to_neg(array):
    """ Map all 0 to -1 """
    ret = np.ones(len(array))
    for i in range(len(array)):
        if(array[i] == 0):
            ret[i] = -1
    return ret

In [121]:
# Create train features and train predictions
all_tweets = tweet_neg.append(tweet_pos)
all_tweets = all_tweets.reset_index().drop(['index', 'tweet'], axis = 1)
x_train = np.vstack(all_tweets.values[:, 0])
y_train = neg_to_zero(all_tweets.values[:, 1])

In [202]:
def clean_data(array):
    """ Clean the data """
    ret = np.zeros(len(array))
    for i in range(len(array)):
        drop_id = len(str(i+1)) + 1
        array[i, 0] = array[i, 0][int(drop_id):]
    return array

In [241]:
# Load test set 
tweets_test = pd.read_csv('data/test_data.txt', sep = '\t', header = None)
tweets_test.head()

,0
0,"1,sea doo pro sea scooter ( sports with the po..."
1,"2,<user> shucks well i work all week so now i ..."
2,"3,i cant stay away from bug thats my baby"
3,"4,<user> no ma'am ! ! ! lol im perfectly fine ..."
4,"5,whenever i fall asleep watching the tv , i a..."


In [242]:
tweets_tst = clean_data(tweets_test.values)

In [207]:
# Map all tweets to the corresponding vector
x_test = [tweet_to_vect(str(x)) for x in tweets_tst]

In [122]:
# Create the model using logistic regression and predict the values
logistic = LogisticRegression(solver = 'lbfgs')
model = logistic.fit(x_train, y_train)
x_test = np.vstack(x_test)
y_test = model.predict(x_test)
y_pred = zero_to_neg(y_test)

In [227]:
def build_submission(y_pred):
    ret = np.ones((len(y_pred), 2))
    for i in range(len(y_pred)):
        ret[i] = np.array([i+1, y_pred[i]])
    return ret

In [232]:
# Build the submission and save it
submission = build_submission(y_pred).astype(int)
sub = pd.DataFrame(data = submission)
sub.columns = ['Id', 'Prediction']
sub.to_csv('predictions/pred1.csv', index=None)